In [179]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib

pd.set_option('display.max_columns', None)


In [180]:
## fantrax rosters

fx = r"C:\Users\mikej\Desktop\fantrax\fx_wk9.csv"

teams = pd.read_csv(fx ,usecols=['Player','Status','Roster Status'])
teams.columns = ['player','team','active_reserve']
teams_dict = {'919':'Philly919','u_c':'unit_circle','NT 8':'Sneads Foot','NT 4':'New Team 4','txms':'txmoonshine','MG':'Team Gamble','grrr':'Putt Pirates','[AW]':'AlphaWired'}
teams['team'] = teams.team.map(teams_dict)
teams = teams.loc[teams.active_reserve=='Active'].set_index('player')

In [181]:
## live scoring feed

file = "https://feeds.datagolf.com/preds/live-tournament-stats?stats=sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_bs,sg_total,distance,accuracy,gir,prox_fw,prox_rgh,scrambling&round=event_avg&display=value&file_format=csv&key=e297e933c3ad47d71ec1626c299e"

live = round(pd.read_csv(file),2).rename(columns={'player_name':'player'})#.set_index('player')

names = live['player'].str.split(expand=True)
names[0] = names[0].str.rstrip(",")
names[1] = names[1].str.rstrip(",")
names['player'] = names[1] + " " + names[0]
names['player'] = np.where(names['player']=='Min Lee', 'Min Woo Lee', names['player'])
names['player'] = np.where(names['player']=='Byeong An', 'Byeong Hun An', names['player'])
names['player'] = np.where(names['player']=='Rooyen Van', 'Erik Van Rooyen', names['player'])
live = live.set_index(names.player)

In [182]:
live.columns

Index(['event_name', 'last_updated', 'stat_display', 'position', 'player',
       'dg_id', 'stat_round', 'total', 'round', 'thru', 'sg_putt', 'sg_arg',
       'sg_app', 'sg_ott', 'sg_t2g', 'sg_bs', 'sg_total', 'distance',
       'accuracy', 'gir', 'prox_fw', 'prox_rgh', 'scrambling'],
      dtype='object')

In [183]:
team_color={
                "Philly919": 'rgb(127,60,141)',
                "unit_circle": 'rgb(17,165,121)',
                "AlphaWired": 'rgb(57,105,172)',
                "Sneads Shoe": 'rgb(242,183,1)',
                "New Team 4": 'rgb(231,63,116)',
                "Team Gamble": 'rgb(230,131,16)',
                "txmoonshine": 'rgb(0,134,139)',
                "Putt Pirates": 'rgb(165,170,153)'}

In [184]:
### opponent inputs ###
current_week = 9

# live look at all 48 active players with player as index
live_merged = pd.merge(teams, live, how='left', left_index=True, right_index=True).fillna(0).sort_values('total')
live_merged_copy = live_merged.copy()
live_merged[['total','round','thru']] = live_merged[['total','round','thru']].astype('int')#.rename(columns={'position':'Pos','total':'Total','round':'Round','thru':'Thru'})

# live_merged = (
#     live_merged
# #     .reset_index()
#     .rename(columns={
#         'player':'Player','team':'Team','position':'Pos','total':'Total','round':'Round','thru':'Thru Cut'}
#            )
# )

def highlight_rows(row):
    value = row.loc['team']
    if value == 'unit_circle':
        color = '#FFCCE5' # Pink
    elif value == 'Philly919':
        color = '#7f3c8d' # Purple
    elif value == 'AlphaWired':
        color = '#3969ac' # Blue
    elif value == 'Sneads Foot':
        color = '#f2b701' # Gold
    elif value == 'New Team 4':
        color = '#e73f74' # Magenta
    elif value == 'Team Gamble':
        color = '#e68310' # Orange
    elif value == 'txmoonshine':
        color = '#00868b' # Aqua
    else:
        color = '#a5aa99' # Grey
    return ['background-color: {}'.format(color) for r in row]


live_merged['holes_remaining'] = (72 - (live_merged['thru']).fillna(0))
live_merged['holes_remaining'] = np.where(live_merged['position']=='CUT',0,live_merged['holes_remaining']).astype('int')

live_merged[['player','team','position','total','round','thru']].style.apply(highlight_rows, axis=1).hide()

player,team,position,total,round,thru
"Lee, Min Woo",unit_circle,T2,-14,-4,18
"Van Rooyen, Erik",New Team 4,T2,-14,-8,18
"Lowry, Shane",unit_circle,T4,-13,0,18
"Knapp, Jake",unit_circle,T4,-13,-5,18
"Young, Cameron",New Team 4,T4,-13,-5,18
"Mitchell, Keith",Putt Pirates,T9,-12,-6,18
"Horschel, Billy",Putt Pirates,T9,-12,-5,18
"An, Byeong Hun",Team Gamble,T21,-10,-6,18
"Ryder, Sam",Sneads Foot,T21,-10,-5,18
"McIlroy, Rory",Sneads Foot,T21,-10,-3,18


In [193]:
team_score = live_merged.groupby('team')[['total']].sum()

thru_cut = pd.DataFrame(live_merged[live_merged.position !='CUT']['team'].value_counts())

df_holes_remaining = pd.DataFrame(live_merged.groupby('team')['holes_remaining'].sum())

table = pd.merge(thru_cut,df_holes_remaining, left_index=True, right_index=True)
table = table.merge(team_score, left_index=True, right_index=True).rename(columns={'team':'Thru Cut','holes_remaining':'Holes Remaining','total':'Team Score'})
# table = pd.DataFrame(table[['total','holes_remaining','team']])
table

,Thru Cut,Holes Remaining,Team Score
unit_circle,6,360,-47
txmoonshine,6,342,-16
AlphaWired,6,432,0
Sneads Foot,5,270,-35
Team Gamble,4,216,-30
New Team 4,3,162,-23
Putt Pirates,3,162,-31
Philly919,3,162,-15


In [149]:
print(f"unit_circle has {live_merged[live_merged.team=='unit_circle']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='unit_circle'][['position','total','round','thru','sg_putt','sg_arg','sg_app','sg_ott','sg_t2g','gir']]

unit_circle has 360 Holes Remaining


,position,total,round,thru,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,gir
player,,,,,,,,,,
Min Woo Lee,T2,-14,-4,18,0.24,0.33,1.33,0.52,2.17,0.64
Shane Lowry,T4,-13,0,18,0.83,-0.22,1.49,0.06,1.34,0.81
Jake Knapp,T4,-13,-5,18,0.37,-0.04,0.76,1.08,1.80,0.71
Corey Conners,T41,-7,-5,18,-0.61,0.18,0.81,0.28,1.27,0.76
Tommy Fleetwood,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
Patrick Cantlay,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00


In [147]:
# live look at all 48 active players with player as index

print(f"Philly919 has {live_merged[live_merged.team=='Philly919']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='Philly919'][['position','total','round','thru','sg_putt','sg_arg','sg_app','sg_ott','sg_t2g','gir']]

Philly919 has 162 Holes Remaining


,position,total,round,thru,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,gir
player,,,,,,,,,,
Chris Kirk,T28,-9,-6,18,-0.31,-0.20,0.95,0.74,1.48,0.76
Russell Henley,T41,-7,1,18,1.79,-0.33,-0.96,0.17,-1.12,0.69
Justin Rose,T64,-1,-1,18,0.42,0.08,-0.80,-0.53,-1.25,0.61
Luke List,CUT,0,0,0,-0.37,1.12,-0.16,-1.12,-0.16,0.64
Sungjae Im,CUT,1,0,0,-0.25,0.03,-0.76,-0.06,-0.78,0.61
Daniel Berger,CUT,1,0,0,-2.51,0.48,0.74,0.24,1.47,0.67


In [150]:
# live look at all 48 active players with player as index

print(f"Team Gamble has {live_merged[live_merged.team=='Team Gamble']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='Team Gamble'][['position','total','round','thru','sg_putt','sg_arg','sg_app','sg_ott','sg_t2g','gir']]

Team Gamble has 216 Holes Remaining


,position,total,round,thru,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,gir
player,,,,,,,,,,
Byeong Hun An,T21,-10,-6,18,0.17,-0.19,0.79,0.64,1.24,0.71
Matthieu Pavon,T28,-9,-1,18,0.83,-0.24,0.42,0.16,0.34,0.71
Tom Hoge,T28,-9,-3,18,0.30,-0.95,1.20,0.60,0.86,0.74
Chesson Hadley,T35,-8,-2,18,0.86,0.38,-0.39,0.07,0.06,0.65
Stephan Jaeger,CUT,-1,0,0,-1.62,0.27,-0.36,1.68,1.59,0.75
Eric Cole,CUT,7,0,0,-0.77,-0.55,-2.83,0.11,-3.27,0.64


In [151]:
# live look at all 48 active players with player as index

print(f"New Team 4 has {live_merged[live_merged.team=='New Team 4']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='New Team 4'][['position','total','round','thru','sg_putt','sg_arg','sg_app','sg_ott','sg_t2g','gir']]

New Team 4 has 162 Holes Remaining


,position,total,round,thru,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,gir
player,,,,,,,,,,
Erik Van Rooyen,T2,-14,-8,18,1.96,-0.21,0.23,0.44,0.46,0.75
Cameron Young,T4,-13,-5,18,0.98,0.26,0.52,0.41,1.19,0.79
Justin Suh,CUT,-1,0,0,-0.91,0.76,0.12,-0.02,0.88,0.61
Thorbjorn Olesen,CUT,1,0,0,-1.10,0.57,-0.40,-0.11,0.05,0.58
Camilo Villegas,T67,1,-3,18,0.15,-0.34,0.36,-1.51,-1.49,0.72
Akshay Bhatia,CUT,3,0,0,-2.39,-0.28,0.23,0.41,0.36,0.64


In [162]:
active_rosters = []

for team in teams.team.unique():
    one_roster = teams.loc[(teams.team==team) & (teams.active_reserve=='Active')].index
    active_rosters.append(one_roster)

cols = teams.team.unique()
active_rosters = pd.DataFrame(active_rosters, index=cols).T
active_rosters

,AlphaWired,unit_circle,Team Gamble,txmoonshine,Philly919,Sneads Foot,New Team 4,Putt Pirates
0,Sam Burns,Patrick Cantlay,Eric Cole,Tom Kim,Luke List,Rory McIlroy,Cameron Young,Keith Mitchell
1,Xander Schauffele,Tommy Fleetwood,Byeong Hun An,J.T. Poston,Sungjae Im,Rickie Fowler,Erik Van Rooyen,Christiaan Bezuidenhout
2,Jordan Spieth,Min Woo Lee,Stephan Jaeger,Matthew Fitzpatrick,Russell Henley,Sam Ryder,Akshay Bhatia,Billy Horschel
3,Jason Day,Corey Conners,Matthieu Pavon,Mark Hubbard,Chris Kirk,Cameron Champ,Justin Suh,Ryo Hisatsune
4,Brian Harman,Shane Lowry,Tom Hoge,Davis Thompson,Justin Rose,Nick Dunlap,Camilo Villegas,Carson Young
5,Harris English,Jake Knapp,Chesson Hadley,Maverick McNealy,Daniel Berger,Parker Coody,Thorbjorn Olesen,Rasmus Hojgaard
